Just run all these cells

# Imports

In [1]:
'''!pip install nnsight -q
!pip install einops -q
!pip install ipywidgets -q
!pip install hf_transfer -q
!pip install plotly -q
!pip install anywidget -q
# Ensure widget extensions are enabled
!jupyter nbextension enable --py widgetsnbextension --sys-prefix 2>/dev/null || true'''

'!pip install nnsight -q\n!pip install einops -q\n!pip install ipywidgets -q\n!pip install hf_transfer -q\n!pip install plotly -q\n!pip install anywidget -q\n# Ensure widget extensions are enabled\n!jupyter nbextension enable --py widgetsnbextension --sys-prefix 2>/dev/null || true'

In [16]:
!jupyter nbextension enable --py widgetsnbextension --sys-prefix 2>/dev/null || true

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [1]:
from nnsight import LanguageModel
from einops import einsum
import torch
import ipywidgets as widgets
from IPython.display import display
import time
import numpy as np
import nnsight

In [2]:
# Enable widget display in Jupyter
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

# This ensures widgets are properly rendered
display(HTML("<script>console.log('Widgets initialized')</script>"))

# Load model

In [3]:
device = 'cuda'

In [4]:
weight_tensor = torch.load('rollouts-big/beta_torch.pt').to(torch.bfloat16)
model_name = 'Qwen/Qwen3-4B'

In [5]:
# Disable compilation warnings since we use dynamic control flow
model = LanguageModel(model_name, device_map=device, dtype=torch.bfloat16, dispatch=True)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
def get_ema_preds(log_preds, alpha=0.5):
    given_alpha = alpha
    preds_list = log_preds.tolist()
    
    ema_preds = []
    cur_ema = None
    for i,pred in enumerate(preds_list):
        # Use a smooth transition from 0.5 to given_alpha, reaching given_alpha at 200 tokens
        alpha = given_alpha
        if cur_ema is None:
            cur_ema = pred
        else:
            cur_ema = alpha*(cur_ema-1) + (1-alpha)*pred #-1 because we have stepped one token
        ema_preds.append(cur_ema)
    return ema_preds
def get_log_preds(activation, weight_tensor):
    return einsum(activation, weight_tensor, 'seq d_model, d_model -> seq')

# Vibe coded UIs

In [8]:
# Create input text box for prompt
prompt_input = widgets.Textarea(
    value="",
    placeholder='Enter your prompt here...',
    description='Prompt:',
    layout=widgets.Layout(width='100%', height='80px')
)

# Create submit button
submit_button = widgets.Button(
    description='Generate Text',
    button_style='success',
    tooltip='Click to start text generation',
    icon='play'
)

# Create progress bar widget
progress_bar = widgets.FloatProgress(
    value=0,
    min=0,
    max=100,
    description='Progress:',
    bar_style='info',
    style={'bar_color': '#20B2AA'},
    orientation='horizontal'
)

# Create percentage label
percentage_label = widgets.HTML(
    value="<b>0.0%</b>",
    description='',
)

# Create horizontal box for progress bar and percentage
progress_row = widgets.HBox([progress_bar, percentage_label])

# Create text widget for token display
token_display = widgets.HTML(
    value="<b>Generated tokens will appear here...</b>",
    placeholder='',
    description='',
)

# Create container for the widgets
progress_container = widgets.VBox([
    widgets.HTML("<h3>Text Generation Progress</h3>"),
    prompt_input,
    submit_button,
    progress_row,
    token_display
])

# Display the widget
display(progress_container)

def on_submit_clicked(b):
    # Reset progress
    progress_bar.value = 0
    percentage_label.value = "<b>0.0%</b>"
    token_display.value = "<b>Generating...</b>"
    
    # Get prompt from input
    prompt = prompt_input.value
    # Apply chat template
    prompt = model.tokenizer.apply_chat_template([{"role": "user", "content": prompt}], tokenize=False, add_generation_prompt=True)
    cur_log_preds = []
    n_tokens_generated = 0
    generated_tokens = []

    with model.generate(prompt, max_new_tokens=32768, do_sample=True) as tracer:
        # Call .all() to apply intervention to each new token
        with tracer.all():
            activations = model.model.layers[15].output[0]
            if len(activations.shape) == 1:
                activations = activations.unsqueeze(0)
            preds = get_log_preds(activations, weight_tensor).tolist()
            if len(preds) > 1:
                pass
            else:
                cur_log_preds+=preds
                ema_preds = get_ema_preds(torch.tensor(cur_log_preds))
                n_tokens_generated+=1
                pred_tokens_remaining = ema_preds[-1]
                pred_percent_through = n_tokens_generated/(n_tokens_generated + pred_tokens_remaining)
                
                token = model.lm_head.output.argmax(dim=-1).tolist()
                token_str = model.tokenizer.decode(token[0][0], skip_special_tokens=False)
                generated_tokens.append(token_str)
                
                # Update progress bar
                progress_bar.value = pred_percent_through * 100
                
                # Update percentage label
                percentage_label.value = f"<b>{pred_percent_through*100:.1f}%</b>"
                
                # Update token display with all generated tokens
                tokens_html = " ".join([f"<span style='background-color: #e6f3ff; padding: 2px 4px; margin: 1px; border-radius: 3px;'>{token}</span>" for token in generated_tokens])
                token_display.value = f"<b>Generated tokens:</b><br>{tokens_html}<br><br><b>Latest:</b> '{token_str}' | <b>Predicted:</b> {pred_percent_through*100:.1f}% through"

# Connect button click to function
submit_button.on_click(on_submit_clicked)


In [29]:
# Create input text box for prompt
prompt_input = widgets.Textarea(
    value="",
    placeholder='Enter your prompt here...',
    description='Prompt:',
    layout=widgets.Layout(width='100%', height='80px')
)

# Create submit button
submit_button = widgets.Button(
    description='Generate Text',
    button_style='success',
    tooltip='Click to start text generation',
    icon='play'
)

# Create progress bar widget
progress_bar = widgets.FloatProgress(
    value=0,
    min=0,
    max=100,
    description='Progress:',
    bar_style='info',
    style={'bar_color': '#20B2AA'},
    orientation='horizontal'
)

# Create percentage label
percentage_label = widgets.HTML(
    value="<b>0.0%</b>",
    description='',
)

# Create horizontal box for progress bar and percentage
progress_row = widgets.HBox([progress_bar, percentage_label])

# Create text widget for token display
token_display = widgets.HTML(
    value="<b>Generated tokens will appear here...</b>",
    placeholder='',
    description='',
)

# Create output widget for the graph
graph_output = widgets.Output()

# Create container for the widgets - graph now above token display
progress_container = widgets.VBox([
    widgets.HTML("<h3>Text Generation Progress</h3>"),
    prompt_input,
    submit_button,
    progress_row,
    graph_output,
    token_display
])

# Display the widget
display(progress_container)

# Initialize FigureWidget outside the loop (Option 2)
import plotly.graph_objects as go
fig_widget = None

def get_color_for_change(change, max_change=800.0):
    """
    Get color based on change in predictions.
    Positive change (increase) -> Red
    Negative change (decrease) -> Green
    """
    # Normalize change to [-1, 1] range
    normalized = max(min(change / max_change, 1.0), -1.0)
    
    if normalized > 0:  # Increase - Red
        # Interpolate from light to dark red
        intensity = int(255 * (1 - normalized * 0.7))  # 255 to ~77
        return f'rgb(255, {intensity}, {intensity})'
    else:  # Decrease - Green
        # Interpolate from light to dark green
        intensity = int(255 * (1 + normalized * 0.7))  # 255 to ~77
        return f'rgb({intensity}, 255, {intensity})'

def calculate_ema(values, alpha=0.2):
    """Calculate exponential moving average
    Lower alpha = smoother (e.g., 0.1-0.3 for good smoothing)
    Higher alpha = follows data more closely (e.g., 0.8-1.0)
    """
    if not values:
        return []
    ema = [values[0]]
    for val in values[1:]:
        ema.append(alpha * val + (1 - alpha) * ema[-1])
    return ema

def on_submit_clicked(b):
    import html
    global fig_widget
    
    # Reset progress
    progress_bar.value = 0
    percentage_label.value = "<b>0.0%</b>"
    token_display.value = "<b>Generating...</b>"
    
    # Initialize FigureWidget once with both traces
    with graph_output:
        graph_output.clear_output(wait=True)
        fig_widget = go.FigureWidget()
        # Add EMA trace first (will be behind)
        fig_widget.add_trace(go.Scatter(
            x=[],
            y=[],
            mode='lines',
            name='EMA (smoothed)',
            line=dict(color='rgba(255, 100, 100, 0.6)', width=3),
            showlegend=True
        ))
        # Add main trace on top
        fig_widget.add_trace(go.Scatter(
            x=[],
            y=[],
            mode='lines',
            name='Predicted Tokens Remaining',
            line=dict(color='rgba(32, 178, 170, 0.8)', width=2),
            showlegend=True
        ))
        fig_widget.update_layout(
            title='Predicted Tokens Remaining Over Time',
            xaxis_title='Token Number',
            yaxis_title='Predicted Tokens Remaining',
            height=300,
            margin=dict(l=50, r=20, t=40, b=40),
            transition_duration=0,
            legend=dict(x=0.7, y=1, bgcolor='rgba(255,255,255,0.8)')
        )
        display(fig_widget)
    
    # Get prompt from input
    prompt = prompt_input.value
    # Apply chat template
    prompt = model.tokenizer.apply_chat_template([{"role": "user", "content": prompt}], tokenize=False, add_generation_prompt=True)
    raw_preds = []  # Store raw predictions without EMA
    cur_log_preds = []  # Store predictions after get_ema_preds (for progress bar)
    generated_tokens = []
    token_changes = []  # Track changes for color coding

    with model.generate(prompt, max_new_tokens=32768, do_sample=True) as tracer:
        # Call .all() to apply intervention to each new token
        with tracer.all():
            activations = model.model.layers[15].output[0]
            if activations.shape[0] == 1:
                if len(raw_preds) > 0:
                    preds = get_log_preds(activations, weight_tensor)
                    exp_preds = torch.exp(preds).item()

                    raw_preds.append(exp_preds)
                
                    cur_log_preds = calculate_ema(raw_preds, alpha=0.2)
                
                    change = cur_log_preds[-1] - cur_log_preds[-2]
                    token_changes.append(change)

                    token = model.lm_head.output.argmax(dim=-1).tolist()
                    token_str = model.tokenizer.decode(token[0][0], skip_special_tokens=False)
                    generated_tokens.append(token_str)
            
                    pred_percent_through = len(raw_preds) / (len(raw_preds) + cur_log_preds[-1]) * 100
                    progress_bar.value = pred_percent_through
            
                    percentage_label.value = f"<b>{pred_percent_through:.1f}%</b>"
            
                    start_idx = max(0, len(generated_tokens) - 2000)
                    display_tokens = generated_tokens[start_idx:]
                    display_changes = token_changes[start_idx:]
                
                    # Calculate dynamic max_change based on actual data
                    max_change = max(abs(c) for c in token_changes) if token_changes else 800.0
                    max_change = max(max_change, 100.0)  # Ensure a minimum threshold
                
                    tokens_html = " ".join([
                        f"<span style='background-color: {get_color_for_change(change, max_change)}; padding: 2px 4px; margin: 1px; border-radius: 3px;' title='Token #{start_idx + i + 1} | Change: {change:+.2f}'>{html.escape(token)}</span>" 
                        for i, (token, change) in enumerate(zip(display_tokens, display_changes))
                    ])
                
                    # Add indicator if tokens are truncated
                    truncated_msg = f"<i>(Showing last 2000 of {len(generated_tokens)} tokens)</i><br>" if len(generated_tokens) > 2000 else ""
                
                    token_display.value = f"<b>Generated tokens:</b> <span style='font-size: 0.9em;'>(🔴 increase / 🟢 decrease)</span><br>{truncated_msg}{tokens_html}<br><br><b>Latest:</b> '{html.escape(token_str)}' | <b>Predicted:</b> {pred_percent_through:.1f}% through"
                
                    if len(cur_log_preds) % 10 == 0 or len(cur_log_preds) == 1:
                        x_values = list(range(1, len(cur_log_preds) + 1))
                    
                        with fig_widget.batch_update():
                            # Trace 0 is EMA (red line)
                            fig_widget.data[0].x = x_values
                            fig_widget.data[0].y = cur_log_preds
                            # Trace 1 is raw predictions (teal line)
                            fig_widget.data[1].x = x_values
                            fig_widget.data[1].y = raw_preds
                
                else:
                    preds = get_log_preds(activations, weight_tensor)
                    exp_preds = torch.exp(preds).item()

                    raw_preds.append(exp_preds)

                
submit_button.on_click(on_submit_clicked)

In [7]:
# Create input text box for prompt
prompt_input = widgets.Textarea(
    value="",
    placeholder='Enter your prompt here...',
    description='Prompt:',
    layout=widgets.Layout(width='100%', height='80px')
)

# Create submit button
submit_button = widgets.Button(
    description='Generate Text',
    button_style='success',
    tooltip='Click to start text generation',
    icon='play'
)

# Create progress bar widget
progress_bar = widgets.FloatProgress(
    value=0,
    min=0,
    max=100,
    description='Progress:',
    bar_style='info',
    style={'bar_color': '#20B2AA'},
    orientation='horizontal'
)

# Create percentage label
percentage_label = widgets.HTML(
    value="<b>0.0%</b>",
    description='',
)

# Create horizontal box for progress bar and percentage
progress_row = widgets.HBox([progress_bar, percentage_label])

# Create text widget for token display
token_display = widgets.HTML(
    value="<b>Generated tokens will appear here...</b>",
    placeholder='',
    description='',
)

# Create output widget for the graph
graph_output = widgets.Output()

# Create steering strength slider
steering_slider = widgets.FloatSlider(
    value=0.0,
    min=-10.0,
    max=10.0,
    step=0.1,
    description='Steering:',
    tooltip='Positive = steer toward completion, Negative = steer away from completion',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='100%')
)

# Create container for the widgets - graph now above token display
progress_container = widgets.VBox([
    widgets.HTML("<h3>Text Generation Progress with Steering</h3>"),
    prompt_input,
    steering_slider,
    submit_button,
    progress_row,
    graph_output,
    token_display
])

# Display the widget
display(progress_container)

# Initialize FigureWidget outside the loop (Option 2)
import plotly.graph_objects as go
fig_widget = None

def get_color_for_change(change, max_change=800.0):
    """
    Get color based on change in predictions.
    Positive change (increase) -> Red
    Negative change (decrease) -> Green
    """
    # Normalize change to [-1, 1] range
    normalized = max(min(change / max_change, 1.0), -1.0)
    
    if normalized > 0:  # Increase - Red
        # Interpolate from light to dark red
        intensity = int(255 * (1 - normalized * 0.7))  # 255 to ~77
        return f'rgb(255, {intensity}, {intensity})'
    else:  # Decrease - Green
        # Interpolate from light to dark green
        intensity = int(255 * (1 + normalized * 0.7))  # 255 to ~77
        return f'rgb({intensity}, 255, {intensity})'

def calculate_ema(values, alpha=0.2):
    """Calculate exponential moving average
    Lower alpha = smoother (e.g., 0.1-0.3 for good smoothing)
    Higher alpha = follows data more closely (e.g., 0.8-1.0)
    """
    if not values:
        return []
    ema = [values[0]]
    for val in values[1:]:
        ema.append(alpha * val + (1 - alpha) * ema[-1])
    return ema

def on_submit_clicked(b):
    import html
    global fig_widget
    
    # Get steering strength from slider
    steering_strength = steering_slider.value
    
    # Reset progress
    progress_bar.value = 0
    percentage_label.value = "<b>0.0%</b>"
    token_display.value = f"<b>Generating... (Steering: {steering_strength:+.1f})</b>"
    
    # Initialize FigureWidget once with both traces
    with graph_output:
        graph_output.clear_output(wait=True)
        fig_widget = go.FigureWidget()
        # Add EMA trace first (will be behind)
        fig_widget.add_trace(go.Scatter(
            x=[],
            y=[],
            mode='lines',
            name='EMA (smoothed)',
            line=dict(color='rgba(255, 100, 100, 0.6)', width=3),
            showlegend=True
        ))
        # Add main trace on top
        fig_widget.add_trace(go.Scatter(
            x=[],
            y=[],
            mode='lines',
            name='Predicted Tokens Remaining',
            line=dict(color='rgba(32, 178, 170, 0.8)', width=2),
            showlegend=True
        ))
        fig_widget.update_layout(
            title=f'Predicted Tokens Remaining Over Time (Steering: {steering_strength:+.1f})',
            xaxis_title='Token Number',
            yaxis_title='Predicted Tokens Remaining',
            height=300,
            margin=dict(l=50, r=20, t=40, b=40),
            transition_duration=0,
            legend=dict(x=0.7, y=1, bgcolor='rgba(255,255,255,0.8)')
        )
        display(fig_widget)
    
    # Get prompt from input
    prompt = prompt_input.value
    # Apply chat template
    prompt = model.tokenizer.apply_chat_template([{"role": "user", "content": prompt}], tokenize=False, add_generation_prompt=True)
    raw_preds = []  # Store raw predictions without EMA
    cur_log_preds = []  # Store predictions after get_ema_preds (for progress bar)
    generated_tokens = []
    token_changes = []  # Track changes for color coding

    with model.generate(prompt, max_new_tokens=32768, do_sample=True) as tracer:
        # Call .all() to apply intervention to each new token
        with tracer.all():
            # Get the original activations
            original_activations = model.model.layers[15].output[0]
            
            # Apply steering to modify the forward pass (affects generation)
            if steering_strength != 0.0:
                model.model.layers[15].output[0][:] = original_activations + steering_strength * weight_tensor
            
            # Use ORIGINAL activations for prediction (measure effect on unsteered state)
            activations = original_activations
                        
            if activations.shape[0] == 1:
                if len(raw_preds) > 0:
                    preds = get_log_preds(activations, weight_tensor)
                    exp_preds = torch.exp(preds).item()

                    raw_preds.append(exp_preds)
                
                    cur_log_preds = calculate_ema(raw_preds, alpha=0.2)
                
                    change = cur_log_preds[-1] - cur_log_preds[-2]
                    token_changes.append(change)

                    token = model.lm_head.output.argmax(dim=-1).tolist()
                    token_str = model.tokenizer.decode(token[0][0], skip_special_tokens=False)
                    generated_tokens.append(token_str)
            
                    pred_percent_through = len(raw_preds) / (len(raw_preds) + cur_log_preds[-1]) * 100
                    progress_bar.value = pred_percent_through
            
                    percentage_label.value = f"<b>{pred_percent_through:.1f}%</b>"
            
                    start_idx = max(0, len(generated_tokens) - 2000)
                    display_tokens = generated_tokens[start_idx:]
                    display_changes = token_changes[start_idx:]
                
                    # Calculate dynamic max_change based on actual data
                    max_change = max(abs(c) for c in token_changes) if token_changes else 800.0
                    max_change = max(max_change, 100.0)  # Ensure a minimum threshold
                
                    tokens_html = " ".join([
                        f"<span style='background-color: {get_color_for_change(change, max_change)}; padding: 2px 4px; margin: 1px; border-radius: 3px;' title='Token #{start_idx + i + 1} | Change: {change:+.2f}'>{html.escape(token)}</span>" 
                        for i, (token, change) in enumerate(zip(display_tokens, display_changes))
                    ])
                
                    # Add indicator if tokens are truncated
                    truncated_msg = f"<i>(Showing last 2000 of {len(generated_tokens)} tokens)</i><br>" if len(generated_tokens) > 2000 else ""
                
                    token_display.value = f"<b>Generated tokens:</b> <span style='font-size: 0.9em;'>(🔴 increase / 🟢 decrease)</span> | <b>Steering: {steering_strength:+.1f}</b><br>{truncated_msg}{tokens_html}<br><br><b>Latest:</b> '{html.escape(token_str)}' | <b>Predicted:</b> {pred_percent_through:.1f}% through"
                
                    if len(cur_log_preds) % 10 == 0 or len(cur_log_preds) == 1:
                        x_values = list(range(1, len(cur_log_preds) + 1))
                    
                        with fig_widget.batch_update():
                            # Trace 0 is EMA (red line)
                            fig_widget.data[0].x = x_values
                            fig_widget.data[0].y = cur_log_preds
                            # Trace 1 is raw predictions (teal line)
                            fig_widget.data[1].x = x_values
                            fig_widget.data[1].y = raw_preds
                
                else:
                    preds = get_log_preds(activations, weight_tensor)
                    exp_preds = torch.exp(preds).item()

                    raw_preds.append(exp_preds)
                
submit_button.on_click(on_submit_clicked)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
!pip install pandas -q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [7]:
import nnsight

In [6]:
def batch_analyze_steering_effects(
    prompts: list, 
    positive_strength=5.0, 
    negative_strength=-5.0, 
    max_tokens=200,
    top_k_track=100  # Track top-100 tokens by logit difference
):
    """
    Batch analyze which tokens are most affected by steering.
    Uses nnsight's batching to run ALL prompts and ALL conditions in parallel.
    
    This runs 3 conditions x N prompts in a SINGLE batched forward pass:
    - Baseline (no steering) for each prompt
    - Positive steering for each prompt
    - Negative steering for each prompt
    
    Returns detailed analysis of which tokens get their logits boosted/suppressed most.
    """
    
    print(f"🚀 FULLY BATCHED ANALYSIS: {len(prompts)} prompts x 3 conditions = {len(prompts)*3} parallel generations")
    print(f"   Generating {max_tokens} tokens each")
    print(f"   Positive steering: +{positive_strength}, Negative steering: {negative_strength}")
    print(f"   This will maximize GPU utilization! 🔥\n")
    
    # Format all prompts
    formatted_prompts = [
        model.tokenizer.apply_chat_template(
            [{"role": "user", "content": prompt}],
            tokenize=False,
            add_generation_prompt=True
        )
        for prompt in prompts
    ]
    
    # Storage for results
    baseline_logits_lists = []
    positive_logits_lists = []
    negative_logits_lists = []
    
    # SINGLE batched generation for ALL prompts and ALL conditions
    with model.generate(max_new_tokens=max_tokens, do_sample=True) as tracer:
        
        # Batch all baseline runs
        for prompt_idx, formatted_prompt in enumerate(formatted_prompts):
            with tracer.invoke(formatted_prompt):
                baseline_logits_list = list().save()
                baseline_logits_lists.append(baseline_logits_list)
                
                with tracer.all():
                    # No steering - just collect logits
                    logits_baseline = model.lm_head.output[0, -1, :]
                    baseline_logits_list.append(logits_baseline)
        
        # Batch all positive steering runs
        for prompt_idx, formatted_prompt in enumerate(formatted_prompts):
            with tracer.invoke(formatted_prompt):
                positive_logits_list = list().save()
                positive_logits_lists.append(positive_logits_list)
                
                with tracer.all():
                    original_activations = model.model.layers[15].output[0]
                    # Apply positive steering
                    model.model.layers[15].output[0][:] = original_activations + positive_strength * weight_tensor
                    logits_positive = model.lm_head.output[0, -1, :]
                    positive_logits_list.append(logits_positive)
        
        # Batch all negative steering runs
        for prompt_idx, formatted_prompt in enumerate(formatted_prompts):
            with tracer.invoke(formatted_prompt):
                negative_logits_list = list().save()
                negative_logits_lists.append(negative_logits_list)
                
                with tracer.all():
                    original_activations = model.model.layers[15].output[0]
                    # Apply negative steering
                    model.model.layers[15].output[0][:] = original_activations + negative_strength * weight_tensor
                    logits_negative = model.lm_head.output[0, -1, :]
                    negative_logits_list.append(logits_negative)
    
    # Process results
    print(f"\n✅ Batch generation complete!")
    print(f"   Total generations: {len(baseline_logits_lists) + len(positive_logits_lists) + len(negative_logits_lists)}")
    
    # Compute logit differences
    all_logit_diffs_positive = []
    all_logit_diffs_negative = []
    
    for prompt_idx in range(len(prompts)):
        baseline_list = baseline_logits_lists[prompt_idx]
        positive_list = positive_logits_lists[prompt_idx]
        negative_list = negative_logits_lists[prompt_idx]
        
        print(f"   Prompt {prompt_idx + 1}: {len(baseline_list)} tokens generated")
        
        # Compute differences for each token
        for token_idx in range(len(baseline_list)):
            baseline = baseline_list[token_idx]
            positive = positive_list[token_idx]
            negative = negative_list[token_idx]
            
            diff_positive = positive - baseline
            diff_negative = negative - baseline
            
            all_logit_diffs_positive.append(diff_positive)
            all_logit_diffs_negative.append(diff_negative)
    
    print(f"\n{'='*80}")
    print("AGGREGATING RESULTS ACROSS ALL TOKENS...")
    print(f"{'='*80}")
    
    # Stack all differences
    stacked_diffs_positive = torch.stack(all_logit_diffs_positive)
    stacked_diffs_negative = torch.stack(all_logit_diffs_negative)
    
    print(f"Total tokens analyzed: {len(all_logit_diffs_positive)}")
    
    # Compute mean difference per token across all generation steps
    mean_diff_positive = stacked_diffs_positive.mean(dim=0)
    mean_diff_negative = stacked_diffs_negative.mean(dim=0)
    
    # Find top tokens by mean logit increase (positive steering)
    top_upvoted_vals, top_upvoted_ids = torch.topk(mean_diff_positive, k=top_k_track)
    top_upvoted_tokens = [model.tokenizer.decode([idx.item()], skip_special_tokens=False) 
                          for idx in top_upvoted_ids]
    
    # Find top tokens by mean logit decrease (negative steering) 
    top_downvoted_vals, top_downvoted_ids = torch.topk(mean_diff_negative, k=top_k_track, largest=False)
    top_downvoted_tokens = [model.tokenizer.decode([idx.item()], skip_special_tokens=False)
                            for idx in top_downvoted_ids]
    
    # Also check which tokens are SUPPRESSED by positive steering
    top_suppressed_by_pos_vals, top_suppressed_by_pos_ids = torch.topk(mean_diff_positive, k=top_k_track, largest=False)
    top_suppressed_by_pos_tokens = [model.tokenizer.decode([idx.item()], skip_special_tokens=False)
                                    for idx in top_suppressed_by_pos_ids]
    
    # And which are BOOSTED by negative steering
    top_boosted_by_neg_vals, top_boosted_by_neg_ids = torch.topk(mean_diff_negative, k=top_k_track)
    top_boosted_by_neg_tokens = [model.tokenizer.decode([idx.item()], skip_special_tokens=False)
                                 for idx in top_boosted_by_neg_ids]
    
    results = {
        'mean_diff_positive': mean_diff_positive,
        'mean_diff_negative': mean_diff_negative,
        'stacked_diffs_positive': stacked_diffs_positive,
        'stacked_diffs_negative': stacked_diffs_negative,
        'top_upvoted': {
            'token_ids': top_upvoted_ids.cpu().tolist(),
            'tokens': top_upvoted_tokens,
            'mean_logit_increase': top_upvoted_vals.cpu().tolist()
        },
        'top_downvoted': {
            'token_ids': top_downvoted_ids.cpu().tolist(),
            'tokens': top_downvoted_tokens,
            'mean_logit_decrease': top_downvoted_vals.cpu().tolist()
        },
        'top_suppressed_by_positive': {
            'token_ids': top_suppressed_by_pos_ids.cpu().tolist(),
            'tokens': top_suppressed_by_pos_tokens,
            'mean_logit_decrease': top_suppressed_by_pos_vals.cpu().tolist()
        },
        'top_boosted_by_negative': {
            'token_ids': top_boosted_by_neg_ids.cpu().tolist(),
            'tokens': top_boosted_by_neg_tokens,
            'mean_logit_increase': top_boosted_by_neg_vals.cpu().tolist()
        },
        'all_baseline_logits': baseline_logits_lists,
        'all_positive_logits': positive_logits_lists,
        'all_negative_logits': negative_logits_lists,
        'prompts': prompts
    }
    
    return results

In [17]:
def display_steering_token_analysis(results, top_n=20):
    """Display which tokens are most affected by steering"""
    
    print(f"\n{'='*80}")
    print("TOKEN-LEVEL STEERING ANALYSIS")
    print(f"{'='*80}\n")
    
    # 1. Tokens most UPVOTED by positive steering
    print(f"🔼 TOP {top_n} TOKENS BOOSTED BY POSITIVE STEERING:")
    print(f"{'Rank':<6} {'Token':<30} {'Mean Logit Increase':<20}")
    print('-' * 80)
    for i in range(min(top_n, len(results['top_upvoted']['tokens']))):
        token = results['top_upvoted']['tokens'][i]
        increase = results['top_upvoted']['mean_logit_increase'][i]
        print(f"{i+1:<6} {repr(token):<30} {increase:<20.4f}")
    
    # 2. Tokens most DOWNVOTED by negative steering
    print(f"\n🔽 TOP {top_n} TOKENS SUPPRESSED BY NEGATIVE STEERING:")
    print(f"{'Rank':<6} {'Token':<30} {'Mean Logit Decrease':<20}")
    print('-' * 80)
    for i in range(min(top_n, len(results['top_downvoted']['tokens']))):
        token = results['top_downvoted']['tokens'][i]
        decrease = results['top_downvoted']['mean_logit_decrease'][i]
        print(f"{i+1:<6} {repr(token):<30} {decrease:<20.4f}")
    
    # 3. Tokens SUPPRESSED by positive steering (interesting!)
    print(f"\n⬇️  TOP {top_n} TOKENS SUPPRESSED BY POSITIVE STEERING:")
    print(f"{'Rank':<6} {'Token':<30} {'Mean Logit Decrease':<20}")
    print('-' * 80)
    for i in range(min(top_n, len(results['top_suppressed_by_positive']['tokens']))):
        token = results['top_suppressed_by_positive']['tokens'][i]
        decrease = results['top_suppressed_by_positive']['mean_logit_decrease'][i]
        print(f"{i+1:<6} {repr(token):<30} {decrease:<20.4f}")
    
    # 4. Tokens BOOSTED by negative steering (also interesting!)
    print(f"\n⬆️  TOP {top_n} TOKENS BOOSTED BY NEGATIVE STEERING:")
    print(f"{'Rank':<6} {'Token':<30} {'Mean Logit Increase':<20}")
    print('-' * 80)
    for i in range(min(top_n, len(results['top_boosted_by_negative']['tokens']))):
        token = results['top_boosted_by_negative']['tokens'][i]
        increase = results['top_boosted_by_negative']['mean_logit_increase'][i]
        print(f"{i+1:<6} {repr(token):<30} {increase:<20.4f}")
    
    # Create visualization
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Tokens Boosted by Positive Steering',
            'Tokens Suppressed by Negative Steering',
            'Tokens Suppressed by Positive Steering',
            'Tokens Boosted by Negative Steering'
        ),
        specs=[[{"type": "bar"}, {"type": "bar"}],
               [{"type": "bar"}, {"type": "bar"}]]
    )
    
    # Plot 1: Top upvoted by positive
    fig.add_trace(
        go.Bar(
            x=results['top_upvoted']['mean_logit_increase'][:top_n],
            y=[f"{repr(t)[:20]}" for t in results['top_upvoted']['tokens'][:top_n]],
            orientation='h',
            marker=dict(color='green'),
            name='Upvoted'
        ),
        row=1, col=1
    )
    
    # Plot 2: Top downvoted by negative
    fig.add_trace(
        go.Bar(
            x=results['top_downvoted']['mean_logit_decrease'][:top_n],
            y=[f"{repr(t)[:20]}" for t in results['top_downvoted']['tokens'][:top_n]],
            orientation='h',
            marker=dict(color='red'),
            name='Downvoted'
        ),
        row=1, col=2
    )
    
    # Plot 3: Top suppressed by positive
    fig.add_trace(
        go.Bar(
            x=results['top_suppressed_by_positive']['mean_logit_decrease'][:top_n],
            y=[f"{repr(t)[:20]}" for t in results['top_suppressed_by_positive']['tokens'][:top_n]],
            orientation='h',
            marker=dict(color='orange'),
            name='Suppressed (Pos)'
        ),
        row=2, col=1
    )
    
    # Plot 4: Top boosted by negative
    fig.add_trace(
        go.Bar(
            x=results['top_boosted_by_negative']['mean_logit_increase'][:top_n],
            y=[f"{repr(t)[:20]}" for t in results['top_boosted_by_negative']['tokens'][:top_n]],
            orientation='h',
            marker=dict(color='purple'),
            name='Boosted (Neg)'
        ),
        row=2, col=2
    )
    
    fig.update_xaxes(title_text="Mean Logit Change", row=1, col=1)
    fig.update_xaxes(title_text="Mean Logit Change", row=1, col=2)
    fig.update_xaxes(title_text="Mean Logit Change", row=2, col=1)
    fig.update_xaxes(title_text="Mean Logit Change", row=2, col=2)
    
    fig.update_layout(height=800, showlegend=False, title_text="Token-Level Steering Effects")
    fig.show()
    
    # Create distribution plot
    print(f"\n{'='*80}")
    print("CREATING DISTRIBUTION PLOT...")
    print(f"{'='*80}")
    
    fig2 = go.Figure()
    
    # Histogram of all logit changes
    fig2.add_trace(go.Histogram(
        x=results['mean_diff_positive'].cpu().to(torch.float).numpy(),
        name='Positive Steering',
        opacity=0.7,
        nbinsx=100
    ))
    
    fig2.add_trace(go.Histogram(
        x=results['mean_diff_negative'].cpu().to(torch.float).numpy(),
        name='Negative Steering',
        opacity=0.7,
        nbinsx=100
    ))
    
    fig2.update_layout(
        title='Distribution of Mean Logit Changes Across All Tokens',
        xaxis_title='Mean Logit Change',
        yaxis_title='Count',
        barmode='overlay'
    )
    fig2.show()

## Run Batch Analysis

Execute the token-level analysis:

In [13]:
# Define test prompts - ADD MORE for better GPU utilization!
# With A40, you can easily run 20-30 prompts in parallel

testset = [
    "Write a poem about the sea.",
    "Explain the theory of relativity in simple terms.",
    "What are the health benefits of regular exercise?",
    "Describe the process of photosynthesis.",
    "What is the significance of the Renaissance period in history?",
    "How does blockchain technology work?",
    "What are some effective strategies for time management?",
    "Explain the concept of artificial intelligence and its applications.",
    "What are the main causes and effects of climate change?",
    "Describe the structure and function of the human brain.",
    "What are some tips for improving public speaking skills?",
    "Explain the importance of biodiversity in ecosystems.",
    "What are the key principles of effective leadership?",
    "Describe the process of evolution by natural selection.",
    "What are some common challenges faced by entrepreneurs?",
    "Explain the role of government in economic development."
]


# Run the fully batched analysis
# This will run 20 prompts x 3 conditions = 60 parallel generations!
steering_analysis_results = batch_analyze_steering_effects(
    prompts=testset,
    positive_strength=10.0,
    negative_strength=-10.0,
    max_tokens=1500,  # Generate 4000 tokens per prompt
    top_k_track=100   # Track top 100 most affected tokens
)

# Display results
display_steering_token_analysis(steering_analysis_results, top_n=30)

🚀 FULLY BATCHED ANALYSIS: 16 prompts x 3 conditions = 48 parallel generations
   Generating 1500 tokens each
   Positive steering: +10.0, Negative steering: -10.0
   This will maximize GPU utilization! 🔥



NNsightException: 

Traceback (most recent call last):
  File "/root/llm-progress-monitor/venv/lib/python3.11/site-packages/nnsight/intervention/backends/execution.py", line 21, in __call__
    tracer.execute(fn)
  File "/root/llm-progress-monitor/venv/lib/python3.11/site-packages/nnsight/intervention/tracing/tracer.py", line 385, in execute
    self.model.interleave(self.fn, *args, **kwargs)
  File "/root/llm-progress-monitor/venv/lib/python3.11/site-packages/nnsight/modeling/mixins/meta.py", line 76, in interleave
    return super().interleave(fn, *args, **kwargs)
  File "/root/llm-progress-monitor/venv/lib/python3.11/site-packages/nnsight/intervention/envoy.py", line 733, in interleave
    fn(*args, **kwargs)
  File "/root/llm-progress-monitor/venv/lib/python3.11/site-packages/nnsight/modeling/language.py", line 173, in __nnsight_generate__
    output = self._model.generate(*args, streamer=streamer, **kwargs)
  File "/root/llm-progress-monitor/venv/lib/python3.11/site-packages/torch

In [12]:
display_steering_token_analysis(steering_analysis_results, top_n=30)


TOKEN-LEVEL STEERING ANALYSIS

🔼 TOP 30 TOKENS BOOSTED BY POSITIVE STEERING:
Rank   Token                          Mean Logit Increase 
--------------------------------------------------------------------------------
1      ' perhaps'                     4.3750              
2      ' possible'                    4.0000              
3      ' explanation'                 3.7031              
4      ' maybe'                       3.5938              
5      ' possibly'                    3.5938              
6      ' appropriate'                 3.5469              
7      ' proceed'                     3.5156              
8      ' particular'                  3.4844              
9      ' enumeration'                 3.4688              
10     ' various'                     3.3750              
11     ' answer'                      3.3594              
12     ' correct'                     3.3594              
13     ' steps'                       3.3594              
14     ' theref


CREATING DISTRIBUTION PLOT...


## Save Results for Further Analysis

In [ ]:
# Save the results
import pickle
from pathlib import Path

output_dir = Path("steering_analysis")
output_dir.mkdir(exist_ok=True)

# Save the full results
with open(output_dir / "token_steering_analysis.pkl", "wb") as f:
    pickle.dump(steering_analysis_results, f)

# Save a CSV of top affected tokens
import pandas as pd

# Create dataframe for top upvoted tokens
df_upvoted = pd.DataFrame({
    'token': steering_analysis_results['top_upvoted']['tokens'],
    'token_id': steering_analysis_results['top_upvoted']['token_ids'],
    'mean_logit_increase': steering_analysis_results['top_upvoted']['mean_logit_increase']
})
df_upvoted.to_csv(output_dir / "top_upvoted_by_positive.csv", index=False)

# Create dataframe for top downvoted tokens
df_downvoted = pd.DataFrame({
    'token': steering_analysis_results['top_downvoted']['tokens'],
    'token_id': steering_analysis_results['top_downvoted']['token_ids'],
    'mean_logit_decrease': steering_analysis_results['top_downvoted']['mean_logit_decrease']
})
df_downvoted.to_csv(output_dir / "top_downvoted_by_negative.csv", index=False)

# Create dataframe for tokens suppressed by positive steering
df_suppressed = pd.DataFrame({
    'token': steering_analysis_results['top_suppressed_by_positive']['tokens'],
    'token_id': steering_analysis_results['top_suppressed_by_positive']['token_ids'],
    'mean_logit_decrease': steering_analysis_results['top_suppressed_by_positive']['mean_logit_decrease']
})
df_suppressed.to_csv(output_dir / "top_suppressed_by_positive.csv", index=False)

# Create dataframe for tokens boosted by negative steering
df_boosted_neg = pd.DataFrame({
    'token': steering_analysis_results['top_boosted_by_negative']['tokens'],
    'token_id': steering_analysis_results['top_boosted_by_negative']['token_ids'],
    'mean_logit_increase': steering_analysis_results['top_boosted_by_negative']['mean_logit_increase']
})
df_boosted_neg.to_csv(output_dir / "top_boosted_by_negative.csv", index=False)

print("✅ Results saved to:", output_dir)
print(f"  - token_steering_analysis.pkl (full results)")
print(f"  - top_upvoted_by_positive.csv")
print(f"  - top_downvoted_by_negative.csv")
print(f"  - top_suppressed_by_positive.csv")
print(f"  - top_boosted_by_negative.csv")

## Memory-Efficient Large-Scale Analysis

Process 1000+ prompts with token sampling across entire generation without OOM errors.

In [7]:
import json
import gc
from tqdm.auto import tqdm
from pathlib import Path
import pandas as pd

def memory_efficient_batch_analysis(
    dataset_path: str,
    positive_strength=10.0,
    negative_strength=-10.0,
    max_tokens=2000,
    chunk_size=4,  # Process N prompts at a time (adjustable based on VRAM)
    token_sample_rate=0.1,  # Sample 10% of tokens across generation for analysis
    top_k_track=100,
    save_every_n_chunks=1  # Save results after every N chunks (default: every chunk)
):
    """
    Memory-efficient analysis that:
    1. Processes dataset in chunks to avoid OOM
    2. Samples tokens ACROSS THE ENTIRE GENERATION (not just final token)
    3. Accumulates statistics incrementally
    4. Handles large datasets (1000+ prompts)
    5. SAVES RESULTS AFTER EACH CHUNK (incremental checkpointing)
    
    Args:
        dataset_path: Path to JSON dataset
        chunk_size: Number of prompts to process at once (reduce if OOM)
        token_sample_rate: Sample rate for tokens (0.1 = 10% of generated tokens)
        save_every_n_chunks: Save checkpoint after every N chunks (1 = every chunk)
    """
    
    # Setup output directory
    output_dir = Path("steering_analysis")
    output_dir.mkdir(exist_ok=True)
    
    # Create run-specific directory with timestamp
    timestamp = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")
    run_dir = output_dir / f"run_{timestamp}"
    run_dir.mkdir(exist_ok=True)
    
    print(f"📂 Loading dataset from {dataset_path}...")
    with open(dataset_path, 'r') as f:
        dataset = json.load(f)
    
    prompts = [item['instruction'] for item in dataset]
    print(f"   Found {len(prompts)} prompts")
    print(f"   Processing in chunks of {chunk_size} (x3 conditions = {chunk_size*3} parallel gens)")
    print(f"   Sampling {token_sample_rate*100:.0f}% of tokens across entire generation")
    print(f"   Expected samples per prompt: ~{int(max_tokens * token_sample_rate)}")
    print(f"   💾 Results will be saved after every {save_every_n_chunks} chunk(s) to: {run_dir}/")
    
    # Initialize accumulators for running statistics
    # We'll accumulate sum and count to compute mean incrementally
    vocab_size = model.config.vocab_size
    
    # Running sums (we'll compute mean at the end)
    sum_diff_positive = torch.zeros(vocab_size, dtype=torch.float32, device='cpu')
    sum_diff_negative = torch.zeros(vocab_size, dtype=torch.float32, device='cpu')
    count_samples = 0
    
    # Process in chunks
    num_chunks = (len(prompts) + chunk_size - 1) // chunk_size
    
    print(f"\n🚀 Starting {num_chunks} chunks...\n")
    
    for chunk_idx in tqdm(range(num_chunks), desc="Processing chunks"):
        # Get chunk of prompts
        start_idx = chunk_idx * chunk_size
        end_idx = min(start_idx + chunk_size, len(prompts))
        chunk_prompts = prompts[start_idx:end_idx]
        
        print(f"\n{'='*80}")
        print(f"Chunk {chunk_idx + 1}/{num_chunks}: Prompts {start_idx + 1}-{end_idx}")
        print(f"{'='*80}")
        
        # Format prompts
        formatted_prompts = [
            model.tokenizer.apply_chat_template(
                [{"role": "user", "content": prompt}],
                tokenize=False,
                add_generation_prompt=True
            )
            for prompt in chunk_prompts
        ]
        
        # Storage for this chunk
        baseline_logits_lists = []
        positive_logits_lists = []
        negative_logits_lists = []
        
        # Batched generation for this chunk
        with model.generate(max_new_tokens=max_tokens, do_sample=True) as tracer:
            
            # Baseline runs
            for formatted_prompt in formatted_prompts:
                with tracer.invoke(formatted_prompt):
                    baseline_logits_list = list().save()
                    baseline_logits_lists.append(baseline_logits_list)
                    
                    with tracer.all():
                        logits_baseline = model.lm_head.output[0, -1, :]
                        baseline_logits_list.append(logits_baseline)
            
            # Positive steering runs
            for formatted_prompt in formatted_prompts:
                with tracer.invoke(formatted_prompt):
                    positive_logits_list = list().save()
                    positive_logits_lists.append(positive_logits_list)
                    
                    with tracer.all():
                        original_activations = model.model.layers[15].output[0]
                        model.model.layers[15].output[0][:] = original_activations + positive_strength * weight_tensor
                        logits_positive = model.lm_head.output[0, -1, :]
                        positive_logits_list.append(logits_positive)
            
            # Negative steering runs
            for formatted_prompt in formatted_prompts:
                with tracer.invoke(formatted_prompt):
                    negative_logits_list = list().save()
                    negative_logits_lists.append(negative_logits_list)
                    
                    with tracer.all():
                        original_activations = model.model.layers[15].output[0]
                        model.model.layers[15].output[0][:] = original_activations + negative_strength * weight_tensor
                        logits_negative = model.lm_head.output[0, -1, :]
                        negative_logits_list.append(logits_negative)
        
        print(f"✅ Generation complete for chunk {chunk_idx + 1}")
        
        # Process this chunk: sample tokens across entire generation
        for prompt_idx in range(len(chunk_prompts)):
            baseline_list = baseline_logits_lists[prompt_idx]
            positive_list = positive_logits_lists[prompt_idx]
            negative_list = negative_logits_lists[prompt_idx]
            
            num_tokens = len(baseline_list)
            
            # Sample tokens across the generation (not just the end)
            sample_indices = torch.randperm(num_tokens)[:int(num_tokens * token_sample_rate)]
            
            # Accumulate differences for sampled tokens
            for token_idx in sample_indices:
                baseline = baseline_list[token_idx].cpu().to(torch.float32)
                positive = positive_list[token_idx].cpu().to(torch.float32)
                negative = negative_list[token_idx].cpu().to(torch.float32)
                
                # Accumulate sums
                sum_diff_positive += (positive - baseline)
                sum_diff_negative += (negative - baseline)
                count_samples += 1
            
            # Free memory immediately
            del baseline_list, positive_list, negative_list
        
        # Clean up chunk data
        del baseline_logits_lists, positive_logits_lists, negative_logits_lists
        gc.collect()
        torch.cuda.empty_cache()
        
        print(f"   Processed {len(chunk_prompts)} prompts")
        print(f"   Total samples so far: {count_samples:,}")
        
        # Save checkpoint after every N chunks
        if (chunk_idx + 1) % save_every_n_chunks == 0 or (chunk_idx + 1) == num_chunks:
            print(f"\n   💾 Saving checkpoint after chunk {chunk_idx + 1}/{num_chunks}...")
            
            # Compute current statistics
            mean_diff_positive = sum_diff_positive / count_samples
            mean_diff_negative = sum_diff_negative / count_samples
            
            # Save checkpoint
            checkpoint = {
                'sum_diff_positive': sum_diff_positive,
                'sum_diff_negative': sum_diff_negative,
                'count_samples': count_samples,
                'chunks_processed': chunk_idx + 1,
                'prompts_processed': min(end_idx, len(prompts)),
                'config': {
                    'dataset_path': dataset_path,
                    'positive_strength': positive_strength,
                    'negative_strength': negative_strength,
                    'max_tokens': max_tokens,
                    'chunk_size': chunk_size,
                    'token_sample_rate': token_sample_rate
                }
            }
            
            torch.save(checkpoint, run_dir / f"checkpoint_chunk_{chunk_idx + 1:04d}.pt")
            
            # Also save top tokens at this checkpoint
            mean_diff_positive_gpu = mean_diff_positive.to(device)
            mean_diff_negative_gpu = mean_diff_negative.to(device)
            
            top_upvoted_vals, top_upvoted_ids = torch.topk(mean_diff_positive_gpu, k=min(top_k_track, 50))
            top_upvoted_tokens = [model.tokenizer.decode([idx.item()], skip_special_tokens=False) 
                                  for idx in top_upvoted_ids]
            
            df_checkpoint = pd.DataFrame({
                'rank': range(1, len(top_upvoted_tokens) + 1),
                'token': top_upvoted_tokens,
                'token_id': top_upvoted_ids.cpu().tolist(),
                'mean_logit_increase': top_upvoted_vals.cpu().tolist()
            })
            df_checkpoint.to_csv(run_dir / f"top_upvoted_chunk_{chunk_idx + 1:04d}.csv", index=False)
            
            print(f"   ✅ Checkpoint saved: {count_samples:,} samples from {min(end_idx, len(prompts))} prompts")
    
    print(f"\n{'='*80}")
    print("COMPUTING FINAL STATISTICS...")
    print(f"{'='*80}")
    print(f"Total prompts analyzed: {len(prompts)}")
    print(f"Total token samples: {count_samples:,}")
    
    # Compute mean differences
    mean_diff_positive = sum_diff_positive / count_samples
    mean_diff_negative = sum_diff_negative / count_samples
    
    # Move to GPU for topk (faster)
    mean_diff_positive = mean_diff_positive.to(device)
    mean_diff_negative = mean_diff_negative.to(device)
    
    # Find top tokens
    top_upvoted_vals, top_upvoted_ids = torch.topk(mean_diff_positive, k=top_k_track)
    top_upvoted_tokens = [model.tokenizer.decode([idx.item()], skip_special_tokens=False) 
                          for idx in top_upvoted_ids]
    
    top_downvoted_vals, top_downvoted_ids = torch.topk(mean_diff_negative, k=top_k_track, largest=False)
    top_downvoted_tokens = [model.tokenizer.decode([idx.item()], skip_special_tokens=False)
                            for idx in top_downvoted_ids]
    
    top_suppressed_by_pos_vals, top_suppressed_by_pos_ids = torch.topk(mean_diff_positive, k=top_k_track, largest=False)
    top_suppressed_by_pos_tokens = [model.tokenizer.decode([idx.item()], skip_special_tokens=False)
                                    for idx in top_suppressed_by_pos_ids]
    
    top_boosted_by_neg_vals, top_boosted_by_neg_ids = torch.topk(mean_diff_negative, k=top_k_track)
    top_boosted_by_neg_tokens = [model.tokenizer.decode([idx.item()], skip_special_tokens=False)
                                 for idx in top_boosted_by_neg_ids]
    
    results = {
        'mean_diff_positive': mean_diff_positive,
        'mean_diff_negative': mean_diff_negative,
        'num_prompts': len(prompts),
        'num_samples': count_samples,
        'top_upvoted': {
            'token_ids': top_upvoted_ids.cpu().tolist(),
            'tokens': top_upvoted_tokens,
            'mean_logit_increase': top_upvoted_vals.cpu().tolist()
        },
        'top_downvoted': {
            'token_ids': top_downvoted_ids.cpu().tolist(),
            'tokens': top_downvoted_tokens,
            'mean_logit_decrease': top_downvoted_vals.cpu().tolist()
        },
        'top_suppressed_by_positive': {
            'token_ids': top_suppressed_by_pos_ids.cpu().tolist(),
            'tokens': top_suppressed_by_pos_tokens,
            'mean_logit_decrease': top_suppressed_by_pos_vals.cpu().tolist()
        },
        'top_boosted_by_negative': {
            'token_ids': top_boosted_by_neg_ids.cpu().tolist(),
            'tokens': top_boosted_by_neg_tokens,
            'mean_logit_increase': top_boosted_by_neg_vals.cpu().tolist()
        },
        'config': {
            'dataset_path': dataset_path,
            'positive_strength': positive_strength,
            'negative_strength': negative_strength,
            'max_tokens': max_tokens,
            'chunk_size': chunk_size,
            'token_sample_rate': token_sample_rate
        },
        'run_dir': str(run_dir),
        'timestamp': timestamp
    }
    
    # Save final results
    print(f"\n💾 Saving final results to {run_dir}/")
    
    import pickle
    with open(run_dir / "final_results.pkl", "wb") as f:
        pickle.dump(results, f)
    
    # Save final CSVs
    df_upvoted_final = pd.DataFrame({
        'rank': range(1, len(results['top_upvoted']['tokens']) + 1),
        'token': results['top_upvoted']['tokens'],
        'token_id': results['top_upvoted']['token_ids'],
        'mean_logit_increase': results['top_upvoted']['mean_logit_increase']
    })
    df_upvoted_final.to_csv(run_dir / "final_top_upvoted.csv", index=False)
    
    df_downvoted_final = pd.DataFrame({
        'rank': range(1, len(results['top_downvoted']['tokens']) + 1),
        'token': results['top_downvoted']['tokens'],
        'token_id': results['top_downvoted']['token_ids'],
        'mean_logit_decrease': results['top_downvoted']['mean_logit_decrease']
    })
    df_downvoted_final.to_csv(run_dir / "final_top_downvoted.csv", index=False)
    
    df_suppressed_final = pd.DataFrame({
        'rank': range(1, len(results['top_suppressed_by_positive']['tokens']) + 1),
        'token': results['top_suppressed_by_positive']['tokens'],
        'token_id': results['top_suppressed_by_positive']['token_ids'],
        'mean_logit_decrease': results['top_suppressed_by_positive']['mean_logit_decrease']
    })
    df_suppressed_final.to_csv(run_dir / "final_top_suppressed.csv", index=False)
    
    df_boosted_neg_final = pd.DataFrame({
        'rank': range(1, len(results['top_boosted_by_negative']['tokens']) + 1),
        'token': results['top_boosted_by_negative']['tokens'],
        'token_id': results['top_boosted_by_negative']['token_ids'],
        'mean_logit_increase': results['top_boosted_by_negative']['mean_logit_increase']
    })
    df_boosted_neg_final.to_csv(run_dir / "final_top_boosted_neg.csv", index=False)
    
    # Save metadata
    metadata = {
        'timestamp': timestamp,
        'num_prompts': results['num_prompts'],
        'num_samples': results['num_samples'],
        'config': results['config']
    }
    with open(run_dir / "final_metadata.json", "w") as f:
        json.dump(metadata, f, indent=2)
    
    print(f"\n✅ Analysis complete!")
    print(f"   📁 All results saved to: {run_dir}/")
    print(f"   - final_results.pkl (full results)")
    print(f"   - final_top_upvoted.csv")
    print(f"   - final_top_downvoted.csv")
    print(f"   - final_top_suppressed.csv")
    print(f"   - final_top_boosted_neg.csv")
    print(f"   - final_metadata.json")
    print(f"   - checkpoint_chunk_*.pt (incremental checkpoints)")
    print(f"   - top_upvoted_chunk_*.csv (incremental top tokens)")
    
    return results

### Run Large-Scale Analysis

Process the full harmful_train.json dataset (1000+ prompts):

In [ ]:
# Run memory-efficient analysis on full dataset
large_scale_results = memory_efficient_batch_analysis(
    dataset_path="/root/llm-progress-monitor/splits/harmful_train.json",
    positive_strength=10.0,
    negative_strength=-10.0,
    max_tokens=1000,
    chunk_size=4,  # Adjust based on VRAM: 5 for A40, reduce to 3 if OOM
    token_sample_rate=1,  # Sample 10% of tokens across generation
    top_k_track=100
)

# Display results
display_steering_token_analysis(large_scale_results, top_n=50)

📂 Loading dataset from /root/llm-progress-monitor/splits/harmful_train.json...
   Found 260 prompts
   Processing in chunks of 4 (x3 conditions = 12 parallel gens)
   Sampling 100% of tokens across entire generation
   Expected samples per prompt: ~1000
   💾 Results will be saved after every 1 chunk(s) to: steering_analysis/run_20251005_200543/

🚀 Starting 65 chunks...



Processing chunks:   0%|          | 0/65 [00:00<?, ?it/s]

You're using a Qwen2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


You have set `compile_config`, but we are unable to meet the criteria for compilation. Compilation will be skipped.



Chunk 1/65: Prompts 1-4
✅ Generation complete for chunk 1
✅ Generation complete for chunk 1
   Processed 4 prompts
   Total samples so far: 4,000

   💾 Saving checkpoint after chunk 1/65...
   ✅ Checkpoint saved: 4,000 samples from 4 prompts

Chunk 2/65: Prompts 5-8
   Processed 4 prompts
   Total samples so far: 4,000

   💾 Saving checkpoint after chunk 1/65...
   ✅ Checkpoint saved: 4,000 samples from 4 prompts

Chunk 2/65: Prompts 5-8
✅ Generation complete for chunk 2
✅ Generation complete for chunk 2
   Processed 4 prompts
   Total samples so far: 8,000

   💾 Saving checkpoint after chunk 2/65...
   ✅ Checkpoint saved: 8,000 samples from 8 prompts

Chunk 3/65: Prompts 9-12
   Processed 4 prompts
   Total samples so far: 8,000

   💾 Saving checkpoint after chunk 2/65...
   ✅ Checkpoint saved: 8,000 samples from 8 prompts

Chunk 3/65: Prompts 9-12
✅ Generation complete for chunk 3
✅ Generation complete for chunk 3
   Processed 4 prompts
   Total samples so far: 12,000

   💾 Saving c

### Save Large-Scale Results

In [11]:
# Save large-scale results
output_dir = Path("steering_analysis")
output_dir.mkdir(exist_ok=True)

# Save full results
timestamp = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")
results_file = output_dir / f"large_scale_analysis_{timestamp}.pkl"

with open(results_file, "wb") as f:
    pickle.dump(large_scale_results, f)

# Save CSVs
df_upvoted_large = pd.DataFrame({
    'rank': range(1, len(large_scale_results['top_upvoted']['tokens']) + 1),
    'token': large_scale_results['top_upvoted']['tokens'],
    'token_id': large_scale_results['top_upvoted']['token_ids'],
    'mean_logit_increase': large_scale_results['top_upvoted']['mean_logit_increase']
})
df_upvoted_large.to_csv(output_dir / f"large_scale_upvoted_{timestamp}.csv", index=False)

df_downvoted_large = pd.DataFrame({
    'rank': range(1, len(large_scale_results['top_downvoted']['tokens']) + 1),
    'token': large_scale_results['top_downvoted']['tokens'],
    'token_id': large_scale_results['top_downvoted']['token_ids'],
    'mean_logit_decrease': large_scale_results['top_downvoted']['mean_logit_decrease']
})
df_downvoted_large.to_csv(output_dir / f"large_scale_downvoted_{timestamp}.csv", index=False)

df_suppressed_large = pd.DataFrame({
    'rank': range(1, len(large_scale_results['top_suppressed_by_positive']['tokens']) + 1),
    'token': large_scale_results['top_suppressed_by_positive']['tokens'],
    'token_id': large_scale_results['top_suppressed_by_positive']['token_ids'],
    'mean_logit_decrease': large_scale_results['top_suppressed_by_positive']['mean_logit_decrease']
})
df_suppressed_large.to_csv(output_dir / f"large_scale_suppressed_{timestamp}.csv", index=False)

df_boosted_neg_large = pd.DataFrame({
    'rank': range(1, len(large_scale_results['top_boosted_by_negative']['tokens']) + 1),
    'token': large_scale_results['top_boosted_by_negative']['tokens'],
    'token_id': large_scale_results['top_boosted_by_negative']['token_ids'],
    'mean_logit_increase': large_scale_results['top_boosted_by_negative']['mean_logit_increase']
})
df_boosted_neg_large.to_csv(output_dir / f"large_scale_boosted_neg_{timestamp}.csv", index=False)

# Save metadata
metadata = {
    'timestamp': timestamp,
    'num_prompts': large_scale_results['num_prompts'],
    'num_samples': large_scale_results['num_samples'],
    'config': large_scale_results['config']
}

with open(output_dir / f"large_scale_metadata_{timestamp}.json", "w") as f:
    json.dump(metadata, f, indent=2)

print(f"\n✅ Large-scale results saved!")
print(f"   Prompts analyzed: {large_scale_results['num_prompts']:,}")
print(f"   Token samples: {large_scale_results['num_samples']:,}")
print(f"   Files saved to: {output_dir}/")
print(f"   - {results_file.name}")
print(f"   - large_scale_upvoted_{timestamp}.csv")
print(f"   - large_scale_downvoted_{timestamp}.csv")
print(f"   - large_scale_suppressed_{timestamp}.csv")
print(f"   - large_scale_boosted_neg_{timestamp}.csv")
print(f"   - large_scale_metadata_{timestamp}.json")

NameError: name 'pickle' is not defined

## View Checkpoint Progress

You can monitor progress by checking the latest checkpoint files:

In [12]:
# List all checkpoint files from the latest run
import glob

# Find the most recent run directory
run_dirs = sorted(Path("steering_analysis").glob("run_*"))
if run_dirs:
    latest_run = run_dirs[-1]
    print(f"📁 Latest run: {latest_run.name}\n")
    
    # List checkpoint files
    checkpoints = sorted(latest_run.glob("checkpoint_chunk_*.pt"))
    print(f"💾 Checkpoints saved: {len(checkpoints)}")
    for cp in checkpoints[-5:]:  # Show last 5
        checkpoint = torch.load(cp)
        print(f"   {cp.name}: {checkpoint['prompts_processed']} prompts, {checkpoint['count_samples']:,} samples")
    
    # Show latest top tokens
    csv_files = sorted(latest_run.glob("top_upvoted_chunk_*.csv"))
    if csv_files:
        latest_csv = csv_files[-1]
        print(f"\n🔼 Top tokens from {latest_csv.name}:")
        df = pd.read_csv(latest_csv)
        print(df.head(10).to_string(index=False))
else:
    print("No runs found yet. Start the analysis first!")

📁 Latest run: run_20251005_193917

💾 Checkpoints saved: 4
   checkpoint_chunk_0001.pt: 4 prompts, 600 samples
   checkpoint_chunk_0002.pt: 8 prompts, 1,200 samples
   checkpoint_chunk_0003.pt: 12 prompts, 1,800 samples
   checkpoint_chunk_0004.pt: 16 prompts, 2,400 samples

🔼 Top tokens from top_upvoted_chunk_0004.csv:
 rank   token  token_id  mean_logit_increase
    1       ‧    144492            11.971065
    2     XYZ     71089            11.851323
    3       賢    119085            11.642118
    4   reach      5545            11.610363
    5     CSR     62976            11.316730
    6     RSA     45641            11.237325
    7  struct      2036            11.205308
    8     '.'     24361            11.129553
    9  eighth     36377            11.016926
   10  figure      7071            11.014008


## Load and Process Results from Checkpoints

Use these functions to reconstruct full results from saved checkpoint files without re-running the analysis:

In [14]:
def load_checkpoint_results(run_dir_path=None, checkpoint_file=None):
    """
    Load results from checkpoint files.
    
    Args:
        run_dir_path: Path to specific run directory (e.g., "steering_analysis/run_20241005_143022")
                     If None, uses the most recent run
        checkpoint_file: Specific checkpoint file to load (e.g., "checkpoint_chunk_0010.pt")
                        If None, uses the latest checkpoint
    
    Returns:
        Dictionary with full analysis results reconstructed from checkpoint
    """
    
    # Find run directory
    if run_dir_path is None:
        run_dirs = sorted(Path("steering_analysis").glob("run_*"))
        if not run_dirs:
            raise FileNotFoundError("No run directories found in steering_analysis/")
        run_dir = run_dirs[-1]
        print(f"📁 Using latest run: {run_dir.name}")
    else:
        run_dir = Path(run_dir_path)
        if not run_dir.exists():
            raise FileNotFoundError(f"Run directory not found: {run_dir}")
        print(f"📁 Using specified run: {run_dir.name}")
    
    # Find checkpoint file
    if checkpoint_file is None:
        checkpoints = sorted(run_dir.glob("checkpoint_chunk_*.pt"))
        if not checkpoints:
            raise FileNotFoundError(f"No checkpoints found in {run_dir}")
        checkpoint_path = checkpoints[-1]
        print(f"💾 Loading latest checkpoint: {checkpoint_path.name}")
    else:
        checkpoint_path = run_dir / checkpoint_file
        if not checkpoint_path.exists():
            raise FileNotFoundError(f"Checkpoint not found: {checkpoint_path}")
        print(f"💾 Loading specified checkpoint: {checkpoint_path.name}")
    
    # Load checkpoint
    checkpoint = torch.load(checkpoint_path, map_location='cpu')
    
    print(f"\n📊 Checkpoint Info:")
    print(f"   Chunks processed: {checkpoint['chunks_processed']}")
    print(f"   Prompts processed: {checkpoint['prompts_processed']}")
    print(f"   Token samples: {checkpoint['count_samples']:,}")
    print(f"   Configuration: {checkpoint['config']}")
    
    # Compute statistics
    print(f"\n🔢 Computing statistics from checkpoint...")
    sum_diff_positive = checkpoint['sum_diff_positive']
    sum_diff_negative = checkpoint['sum_diff_negative']
    count_samples = checkpoint['count_samples']
    
    mean_diff_positive = sum_diff_positive / count_samples
    mean_diff_negative = sum_diff_negative / count_samples
    
    # Move to GPU for topk
    mean_diff_positive_gpu = mean_diff_positive.to(device)
    mean_diff_negative_gpu = mean_diff_negative.to(device)
    
    # Find top tokens
    top_k = 100
    
    top_upvoted_vals, top_upvoted_ids = torch.topk(mean_diff_positive_gpu, k=top_k)
    top_upvoted_tokens = [model.tokenizer.decode([idx.item()], skip_special_tokens=False) 
                          for idx in top_upvoted_ids]
    
    top_downvoted_vals, top_downvoted_ids = torch.topk(mean_diff_negative_gpu, k=top_k, largest=False)
    top_downvoted_tokens = [model.tokenizer.decode([idx.item()], skip_special_tokens=False)
                            for idx in top_downvoted_ids]
    
    top_suppressed_by_pos_vals, top_suppressed_by_pos_ids = torch.topk(mean_diff_positive_gpu, k=top_k, largest=False)
    top_suppressed_by_pos_tokens = [model.tokenizer.decode([idx.item()], skip_special_tokens=False)
                                    for idx in top_suppressed_by_pos_ids]
    
    top_boosted_by_neg_vals, top_boosted_by_neg_ids = torch.topk(mean_diff_negative_gpu, k=top_k)
    top_boosted_by_neg_tokens = [model.tokenizer.decode([idx.item()], skip_special_tokens=False)
                                 for idx in top_boosted_by_neg_ids]
    
    results = {
        'mean_diff_positive': mean_diff_positive_gpu,
        'mean_diff_negative': mean_diff_negative_gpu,
        'num_prompts': checkpoint['prompts_processed'],
        'num_samples': count_samples,
        'top_upvoted': {
            'token_ids': top_upvoted_ids.cpu().tolist(),
            'tokens': top_upvoted_tokens,
            'mean_logit_increase': top_upvoted_vals.cpu().tolist()
        },
        'top_downvoted': {
            'token_ids': top_downvoted_ids.cpu().tolist(),
            'tokens': top_downvoted_tokens,
            'mean_logit_decrease': top_downvoted_vals.cpu().tolist()
        },
        'top_suppressed_by_positive': {
            'token_ids': top_suppressed_by_pos_ids.cpu().tolist(),
            'tokens': top_suppressed_by_pos_tokens,
            'mean_logit_decrease': top_suppressed_by_pos_vals.cpu().tolist()
        },
        'top_boosted_by_negative': {
            'token_ids': top_boosted_by_neg_ids.cpu().tolist(),
            'tokens': top_boosted_by_neg_tokens,
            'mean_logit_increase': top_boosted_by_neg_vals.cpu().tolist()
        },
        'config': checkpoint['config'],
        'checkpoint_info': {
            'chunks_processed': checkpoint['chunks_processed'],
            'checkpoint_file': checkpoint_path.name,
            'run_dir': str(run_dir)
        }
    }
    
    print(f"✅ Results loaded successfully!")
    
    return results


def list_all_runs():
    """List all available run directories with their metadata"""
    
    run_dirs = sorted(Path("steering_analysis").glob("run_*"))
    
    if not run_dirs:
        print("No run directories found in steering_analysis/")
        return []
    
    print(f"📁 Found {len(run_dirs)} run(s):\n")
    
    run_info = []
    
    for run_dir in run_dirs:
        # Try to load metadata
        metadata_file = run_dir / "final_metadata.json"
        if metadata_file.exists():
            with open(metadata_file, 'r') as f:
                metadata = json.load(f)
            status = "✅ COMPLETE"
        else:
            # Get info from latest checkpoint
            checkpoints = sorted(run_dir.glob("checkpoint_chunk_*.pt"))
            if checkpoints:
                checkpoint = torch.load(checkpoints[-1], map_location='cpu')
                metadata = {
                    'num_prompts': checkpoint['prompts_processed'],
                    'num_samples': checkpoint['count_samples'],
                    'config': checkpoint['config']
                }
                status = f"⏸️  IN PROGRESS ({checkpoint['chunks_processed']} chunks)"
            else:
                continue
        
        info = {
            'run_dir': run_dir.name,
            'path': str(run_dir),
            'status': status,
            'prompts': metadata.get('num_prompts', 'N/A'),
            'samples': metadata.get('num_samples', 'N/A'),
            'config': metadata.get('config', {})
        }
        run_info.append(info)
        
        print(f"  {status}")
        print(f"  Directory: {run_dir.name}")
        print(f"  Prompts: {info['prompts']:,}" if isinstance(info['prompts'], int) else f"  Prompts: {info['prompts']}")
        print(f"  Samples: {info['samples']:,}" if isinstance(info['samples'], int) else f"  Samples: {info['samples']}")
        if info['config']:
            print(f"  Dataset: {info['config'].get('dataset_path', 'N/A')}")
            print(f"  Chunk size: {info['config'].get('chunk_size', 'N/A')}")
            print(f"  Sample rate: {info['config'].get('token_sample_rate', 'N/A')}")
        print()
    
    return run_info


def compare_checkpoints(run_dir_path=None, num_checkpoints=5):
    """
    Compare top tokens across multiple checkpoints to see how results stabilize.
    
    Args:
        run_dir_path: Path to run directory (None = latest)
        num_checkpoints: Number of checkpoints to compare
    """
    
    # Find run directory
    if run_dir_path is None:
        run_dirs = sorted(Path("steering_analysis").glob("run_*"))
        if not run_dirs:
            raise FileNotFoundError("No run directories found")
        run_dir = run_dirs[-1]
    else:
        run_dir = Path(run_dir_path)
    
    print(f"📁 Analyzing run: {run_dir.name}\n")
    
    # Get all checkpoints
    checkpoints = sorted(run_dir.glob("checkpoint_chunk_*.pt"))
    
    if not checkpoints:
        print("No checkpoints found!")
        return
    
    # Sample checkpoints evenly
    if len(checkpoints) <= num_checkpoints:
        selected_checkpoints = checkpoints
    else:
        step = len(checkpoints) // num_checkpoints
        selected_checkpoints = [checkpoints[i * step] for i in range(num_checkpoints)]
        selected_checkpoints.append(checkpoints[-1])  # Always include last
    
    print(f"📊 Comparing {len(selected_checkpoints)} checkpoints:\n")
    
    comparison_data = []
    
    for cp_path in selected_checkpoints:
        cp = torch.load(cp_path, map_location='cpu')
        
        # Compute top tokens
        mean_diff = cp['sum_diff_positive'] / cp['count_samples']
        top_vals, top_ids = torch.topk(mean_diff.to(device), k=10)
        top_tokens = [model.tokenizer.decode([idx.item()], skip_special_tokens=False) 
                     for idx in top_ids]
        
        comparison_data.append({
            'checkpoint': cp_path.name,
            'prompts': cp['prompts_processed'],
            'samples': cp['count_samples'],
            'top_tokens': top_tokens,
            'top_values': top_vals.cpu().tolist()
        })
        
        print(f"  {cp_path.name}")
        print(f"    Prompts: {cp['prompts_processed']}, Samples: {cp['count_samples']:,}")
        print(f"    Top 3 tokens: {', '.join(repr(t) for t in top_tokens[:3])}")
        print()
    
    # Create comparison DataFrame
    print("📈 Token rank changes across checkpoints:\n")
    
    all_tokens = set()
    for item in comparison_data:
        all_tokens.update(item['top_tokens'])
    
    # Track rank of each token
    for token in list(all_tokens)[:10]:  # Show top 10 unique tokens
        ranks = []
        for item in comparison_data:
            if token in item['top_tokens']:
                ranks.append(item['top_tokens'].index(token) + 1)
            else:
                ranks.append('-')
        
        print(f"{repr(token):<20} Ranks: {' -> '.join(str(r) for r in ranks)}")
    
    return comparison_data

### Example Usage

Load and analyze results from saved checkpoints:

In [15]:
# Example 1: List all available runs
print("=" * 80)
print("LISTING ALL RUNS")
print("=" * 80)
list_all_runs()

LISTING ALL RUNS
📁 Found 1 run(s):

  ⏸️  IN PROGRESS (4 chunks)
  Directory: run_20251005_193917
  Prompts: 16
  Samples: 2,400
  Dataset: /root/llm-progress-monitor/splits/harmful_train.json
  Chunk size: 4
  Sample rate: 0.1



[{'run_dir': 'run_20251005_193917',
  'path': 'steering_analysis/run_20251005_193917',
  'status': '⏸️  IN PROGRESS (4 chunks)',
  'prompts': 16,
  'samples': 2400,
  'config': {'dataset_path': '/root/llm-progress-monitor/splits/harmful_train.json',
   'positive_strength': 10.0,
   'negative_strength': -10.0,
   'max_tokens': 1500,
   'chunk_size': 4,
   'token_sample_rate': 0.1}}]

In [19]:
# Example 2: Load latest checkpoint from most recent run
loaded_results = load_checkpoint_results()

# Display the results using existing visualization function
display_steering_token_analysis(loaded_results, top_n=30)

📁 Using latest run: run_20251005_193917
💾 Loading latest checkpoint: checkpoint_chunk_0004.pt

📊 Checkpoint Info:
   Chunks processed: 4
   Prompts processed: 16
   Token samples: 2,400
   Configuration: {'dataset_path': '/root/llm-progress-monitor/splits/harmful_train.json', 'positive_strength': 10.0, 'negative_strength': -10.0, 'max_tokens': 1500, 'chunk_size': 4, 'token_sample_rate': 0.1}

🔢 Computing statistics from checkpoint...
✅ Results loaded successfully!

TOKEN-LEVEL STEERING ANALYSIS

🔼 TOP 30 TOKENS BOOSTED BY POSITIVE STEERING:
Rank   Token                          Mean Logit Increase 
--------------------------------------------------------------------------------
1      '‧'                            11.9711             
2      ' XYZ'                         11.8513             
3      '賢'                            11.6421             
4      ' reach'                       11.6104             
5      ' CSR'                         11.3167             
6      ' RSA'     


CREATING DISTRIBUTION PLOT...


In [ ]:
# Example 3: Load specific checkpoint from specific run
# loaded_results = load_checkpoint_results(
#     run_dir_path="steering_analysis/run_20241005_143022",
#     checkpoint_file="checkpoint_chunk_0050.pt"
# )
# display_steering_token_analysis(loaded_results, top_n=30)

In [ ]:
# Example 4: Compare how top tokens change across checkpoints
comparison = compare_checkpoints(num_checkpoints=5)

### Advanced: Query Specific Tokens

Check how specific tokens are affected by steering:

In [ ]:
def query_token_effects(results, tokens_to_check):
    """
    Check how specific tokens are affected by steering.
    
    Args:
        results: Results dictionary from load_checkpoint_results()
        tokens_to_check: List of token strings to check (e.g., ['!', 'Sorry', '\n\n'])
    """
    
    print(f"\n{'='*80}")
    print("QUERYING SPECIFIC TOKENS")
    print(f"{'='*80}\n")
    
    for token_str in tokens_to_check:
        # Encode token
        token_ids = model.tokenizer.encode(token_str, add_special_tokens=False)
        
        if len(token_ids) == 0:
            print(f"⚠️  Token '{token_str}' could not be encoded")
            continue
        
        if len(token_ids) > 1:
            print(f"⚠️  Token '{token_str}' encodes to multiple IDs: {token_ids}")
            print(f"    Using first ID: {token_ids[0]}")
        
        token_id = token_ids[0]
        
        # Get effects
        positive_effect = results['mean_diff_positive'][token_id].item()
        negative_effect = results['mean_diff_negative'][token_id].item()
        
        # Get rank
        all_positive_effects = results['mean_diff_positive']
        rank_positive = (all_positive_effects > positive_effect).sum().item() + 1
        
        print(f"Token: {repr(token_str)}")
        print(f"  Token ID: {token_id}")
        print(f"  Positive steering effect: {positive_effect:+.4f} (rank #{rank_positive:,} out of {len(all_positive_effects):,})")
        print(f"  Negative steering effect: {negative_effect:+.4f}")
        
        if positive_effect > 0:
            print(f"  → Positive steering BOOSTS this token by {positive_effect:.4f} logits")
        else:
            print(f"  → Positive steering SUPPRESSES this token by {abs(positive_effect):.4f} logits")
        
        print()


# Example usage:
# loaded_results = load_checkpoint_results()
# query_token_effects(loaded_results, ['!', 'Sorry', '\n\n', '.', 'I', 'cannot', 'refuse'])